In [ ]:
import os
import requests
import zipfile
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

In [ ]:
zip_file_url = "https://zenodo.org/record/7548320/files/WCEBleedGen.zip"
download_dir = "/content/data"

In [ ]:
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

In [ ]:
zip_file_path = os.path.join(download_dir, "data.zip")
response = requests.get(zip_file_url, stream=True)
with open(zip_file_path, "wb") as file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            file.write(chunk)
with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
    zip_ref.extractall(download_dir)

In [ ]:
# Specify the path of the folder you want to delete
folder_to_delete = "/content/data/WCEBleedGen/bleeding/Bounding boxes"

# Use shutil.rmtree() to delete the folder and its contents
shutil.rmtree(folder_to_delete)

In [ ]:
import shutil
main_folder = "/content/data/WCEBleedGen/non-bleeding"

# Clean the main folder by moving all files to the main folder and removing subfolders
for subdir, _, files in os.walk(main_folder):
    for file in files:
        src_file = os.path.join(subdir, file)
        dest_file = os.path.join(main_folder, file)
        shutil.move(src_file, dest_file)
    if subdir != main_folder:
        os.rmdir(subdir)


In [ ]:
main_folder = "/content/data/WCEBleedGen"
for subdir, dirs, files in os.walk(main_folder):
    # Check if the .ipynb_checkpoints folder is in the list of directories
    if ".ipynb_checkpoints" in dirs:
        # Construct the path to the .ipynb_checkpoints folder
        checkpoints_folder = os.path.join(subdir, ".ipynb_checkpoints")

        # Delete the .ipynb_checkpoints folder
        shutil.rmtree(checkpoints_folder)

In [ ]:
main_folder = "/content/data"
# Define data transforms and create a dataset:
data_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = os.path.join(main_folder, "WCEBleedGen")
dataset = datasets.ImageFolder(data_dir, transform=data_transform)

In [ ]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and testing sets
train_size = 0.8  # You can adjust the training size as needed
train_dataset, test_dataset = train_test_split(dataset, train_size=train_size, shuffle=True, random_state=42)


In [ ]:
# Create DataLoaders for training and testing
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Load a pre-trained model (e.g., ResNeXt-50)
model = models.resnext50_32x4d(pretrained=True)
num_classes = 2
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/10, Loss: 0.32605257020755246
Epoch 2/10, Loss: 0.08367535261663073
Epoch 3/10, Loss: 0.036242641279276344
Epoch 4/10, Loss: 0.016443793705989126
Epoch 5/10, Loss: 0.014603066769654326
Epoch 6/10, Loss: 0.014534715963897972
Epoch 7/10, Loss: 0.015453059357241727
Epoch 8/10, Loss: 0.005318417123017945
Epoch 9/10, Loss: 0.006182644611478529
Epoch 10/10, Loss: 0.005983044874014784


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 99.24%
